# CSS Codes

## Background


### Stabilizer groups.

Let $G$ be a group and $X$ be a set. A group action $\varphi$ of $G$ on $X$ is a function $\varphi \colon G \times X \to X$ such that:
-  for all $x \in X$, we have $\varphi(\mathrm{id}_G, x) = x$;
-  for all $g,h \in G$, we have $\varphi(gh,x) = \varphi(g,\varphi(h,x))$.

Often we drop the notation $\varphi$ and simply write $gx$ to mean $\varphi(g,x)$.
The stabilizer of the group action of $G$ on an element $x \in X$ is the set:
\begin{equation*}
    G_x = \{ g \in G | gx = x \}.
\end{equation*}
That is, the elements of $G$ that act like the identity on $x$.

**Example.**
Consider the symmetric group $S_3$ acting on the set $X = \{0,1,2\}$ via permutations. Each permutation $\sigma$ can be written using integers $a_0$, $a_1$ and $a_2$ such that $\{a_0,a_1,a_2\} = \{0,1,2\}$ by assigning
\begin{equation*}
    0 \to a_0, \;\; 1 \to a_1, \; \; 2 \to a_2.
\end{equation*}
We can represent this more compactly by a tuple $\sigma = (a_0,a_1,a_2)$ where the $i^{\text{th}}$ entry is the image of $i$ under the permutation $\sigma$. For example, the tuple $(2,1,0)$ sends $0 \to 2$, $1 \to 1$, and $2 \to 0$. Using this representation we can create a Python class for a permutation.

In [ ]:
class Permutation:
    """
    A permutation of {0, ..., n-1}, represented as a tuple A = (a0, a1, ..., an-1)
    where i -> A[i].
    """
    def __init__(self,
                 mapping: tuple[int,...]):
        self.n = len(mapping)
        if set(mapping) != set(range(self.n)):
            raise ValueError("A permutation must contain each number from 0 to n-1 exactly once.")
        self.mapping = mapping

    def __call__(self, x: int):
        """
        Returns the image of x under the permutation.
        Can be called like a function: i.e. p = Permutation(), then p(x) = p.__call__(x).
        """
        if not (x in range(self.n)):
            raise ValueError(f"Permutation only acts on integers 0-n. Got x={x}.")
        return self.mapping[x]
    
    def compose(self, other: 'Permutation') -> 'Permutation':
        """
        Return the composition of two permutations: C = B ∘ A.
        For permutations A = self and B = other, the composition C(i) = B(A(i)).
        """
        if other.n != self.n:
            raise ValueError(f"Permutations of different sets cannot be composed. First is on {self.n}, second is on {other.n}.")
    
        return Permutation(tuple(other(self(i)) for i in range(self.n)))
    
    def fixes(self, i: int) -> bool:
        """
        Returns True if the permutation fixes i and False otherwise.
        """
        if not (0 <= i < self.n):
            raise ValueError(f"Invalid i. Permutation acts on 0 to n-1. Got {i}.")
        return self(i) == i

    def fixed_points(self) -> list[bool]:
        """
        Returns a list of booleans indicating which points are fixed by the permutation.
        The ith entry is True if the permutation fixes the element i, and False otherwise.
        """
        return [self(i) == i for i in range(self.n)]
    
    def __repr__(self) -> str:
        return f"Permutation={self.mapping}."

We wish to consider stabilizers of points in $X$. We first define the elements of the symmetric group $S_3$.

In [ ]:
S3_elements = [(0,1,2),
               (0,2,1),
               (1,0,2),
               (1,2,0),
               (2,0,1),
               (2,1,0)]

S3 = [Permutation(p) for p in S3_elements]

In [ ]:
p = S3[-1]
print(p)
print(f"Permutation {p.mapping} acts as:")
for x in range(3):
    print(f"{x} -> {p(x)}")


There are four Pauli matrices: $I$, $X$, $Y$, and $Z$. These form a group under matrix multiplication where each matrix is of order 2. Thus, we may form tensor groups of Pauli matrices. 


One class of quantum error correction codes can be defined using tensor products of Pauli matrices. In particular, a _stabilizer code_ is the positive eigenspace of an abelian subgroup of tensors of Pauli matrices. There are three non-trivial Pauli matrices: $X$, $Y$, $Z$. An _$X$-type stabilizer_ is a tensor product that consists of $X$ matrices and identity matrices. Similarly, we define a _$Z$-type stabilizer_. A _CSS code_ is a stabilizer code with only $X$-type and $Z$-type generators. 

The 